In [ ]:
import argparse
import torch
import torch.nn as nn
import numpy as np
import os
import pickle
import nltk
import pandas as pd
from torch.nn.utils.rnn import pack_padded_sequence
from torchvision import transforms
from torch.utils.data import DataLoader,Dataset
import csv
from PIL import Image
from caption_model.CNN_RNN import EncoderCNN, DecoderRNN
from tqdm import tqdm
import matplotlib.pyplot as plt
from nltk.translate.bleu_score import sentence_bleu

# Device configurationresul
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')

In [ ]:
params={'image_size':512,
        'lr':2e-4,
        'beta1':0.5,
        'beta2':0.999,
        'batch_size':64,
        'epochs':10000,
        'data_path':'../../data/dataset/coco/',
        'train_csv':'train_annotation.csv',
        'val_csv':'val_annotation.csv',
        'vocab_path':'../../data/dataset/coco/vocab.pkl',
        'embed_size':300,
        'hidden_size':512,
        'num_layers':1,}

In [ ]:
class CustomDataset(Dataset):
    """COCO Custom Dataset compatible with torch.utils.data.DataLoader."""
    def __init__(self, data_path, csv, class_dataset, vocab, transform=None):
        """Set the path for images, captions and vocabulary wrapper.
        
        Args:
            root: image directory.
            json: coco annotation file path.
            vocab: vocabulary wrapper.
            transform: image transformer.
        """
        self.root = data_path+class_dataset+'/'
        self.df = pd.read_csv(data_path+csv)
        self.class_dataset=class_dataset
        self.vocab = vocab
        self.transform = transform

    def __getitem__(self, index):
        """Returns one data pair (image and caption)."""
        df = self.df
        vocab = self.vocab
        img_id=df.loc[index]
        image_path = self.root+img_id['image_file']
        caption=img_id['caption']
        image = Image.open(image_path).convert('RGB')
        if self.transform is not None:
            image = self.transform(image)

        # Convert caption (string) to word ids.
        tokens = nltk.tokenize.word_tokenize(str(caption).lower())
        caption = []
        caption.append(vocab('<start>'))
        caption.extend([vocab(token) for token in tokens])
        caption.append(vocab('<end>'))
        target = torch.Tensor(caption)
        return image, target

    def __len__(self):
        return len(self.df)
    
class Vocabulary(object):
    """Simple vocabulary wrapper."""
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0

    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1

    def __call__(self, word):
        if not word in self.word2idx:
            return self.word2idx['<unk>']
        return self.word2idx[word]

    def __len__(self):
        return len(self.word2idx)

def collate_fn(data):
    """Creates mini-batch tensors from the list of tuples (image, caption).
    
    We should build custom collate_fn rather than using default collate_fn, 
    because merging caption (including padding) is not supported in default.

    Args:
        data: list of tuple (image, caption). 
            - image: torch tensor of shape (3, 256, 256).
            - caption: torch tensor of shape (?); variable length.

    Returns:
        images: torch tensor of shape (batch_size, 3, 256, 256).
        targets: torch tensor of shape (batch_size, padded_length).
        lengths: list; valid length for each padded caption.
    """
    # Sort a data list by caption length (descending order).
    data.sort(key=lambda x: len(x[1]), reverse=True)
    images, captions = zip(*data)

    # Merge images (from tuple of 3D tensor to 4D tensor).
    images = torch.stack(images, 0)

    # Merge captions (from tuple of 1D tensor to 2D tensor).
    lengths = [len(cap) for cap in captions]
    targets = torch.zeros(len(captions), max(lengths)).long()
    for i, cap in enumerate(captions):
        end = lengths[i]
        targets[i, :end] = cap[:end]        
    return images, targets, lengths

def idx2word(vocab, indices):
    sentence = []
    for i in range(params['batch_size']):
        indices[i].cpu().numpy()
    
    for index in indices:
        word = vocab.idx2word[index]
        sentence.append(word)
    return sentence

In [ ]:
with open(params['vocab_path'], 'rb') as f:
        vocab = pickle.load(f)
transform = transforms.Compose([ 
        transforms.Resize(params['image_size']),
        transforms.RandomHorizontalFlip(), 
        transforms.ToTensor(), 
        transforms.Normalize((0.485, 0.456, 0.406), 
                             (0.229, 0.224, 0.225))])

train_dataset=CustomDataset(params['data_path'],params['train_csv'],'train',vocab,transform=transform)
val_dataset=CustomDataset(params['data_path'],params['val_csv'],'val',vocab,transform=transform)
train_dataloader=DataLoader(train_dataset,batch_size=params['batch_size'],shuffle=True,collate_fn=collate_fn)
val_dataloader=DataLoader(val_dataset,batch_size=params['batch_size'],shuffle=True,collate_fn=collate_fn)

In [ ]:
encoder = EncoderCNN(params['embed_size']).to(device)
decoder = DecoderRNN(params['embed_size'], params['hidden_size'], len(vocab), params['num_layers']).to(device)
criterion = nn.CrossEntropyLoss()
model_param = list(decoder.parameters()) + list(encoder.linear.parameters()) + list(encoder.bn.parameters())
optimizer = torch.optim.Adam(model_param, lr=params['lr'], betas=(params['beta1'], params['beta2']))

In [6]:
plt_count=0
sum_loss= 1000.0
for epoch in range(params['epochs']):
    train=tqdm(train_dataloader)
    count=0
    train_loss = 0.0
    for images,captions,lengths in train:
        count+=1
        images = images.to(device)
        captions = captions.to(device)
        targets = pack_padded_sequence(captions, lengths, batch_first=True)[0]
        features = encoder(images)
        outputs = decoder(features, captions, lengths)
        outputs = outputs
        loss = criterion(outputs, targets)
        decoder.zero_grad()
        encoder.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss+=loss.item()
        train.set_description(f"epoch: {epoch+1}/{params['epochs']} Step: {count+1} loss : {train_loss/count:.4f} ")
    with torch.no_grad():
        val_count=0
        val_loss = 0.0 
        val_bleu_loss=0.0
        val=tqdm(val_dataloader)
        for images,captions,lengths in val:
            val_count+=1
            images = images.to(device)
            captions = captions.to(device)
            targets = pack_padded_sequence(captions, lengths, batch_first=True)[0]
            features = encoder(images)
            outputs = decoder(features, captions, lengths)
            outputs = outputs
            loss = criterion(outputs, targets)
            val_loss+=loss.item()
            val.set_description(f"epoch: {epoch+1}/{params['epochs']} Step: {val_count+1} loss : {val_loss/val_count:.4f} ")
    if val_loss<sum_loss:
        sum_loss=val_loss
        torch.save(encoder.state_dict(), '../../model/CNN_RNN/encoder_check.pth')
        torch.save(decoder.state_dict(), '../../model/CNN_RNN/decoder_check.pth')
        

epoch: 33/10000 Step: 4493 loss : 1.4371 :  69%|██████▉   | 4492/6471 [1:21:30<54:36,  1.66s/it]  

: 